In [1]:
import gc
from ollama import Client

In [2]:
def clear_memory():
    gc.collect()

In [3]:
def get_users_ingredients():
    users_ingredients = []
    print("Enter the ingredients you have (type 'end' to finish):")
    while True:
        user_input = input("Ingredient: ").strip().lower()
        if user_input == 'end':
            break
        if user_input:
            users_ingredients.append(user_input)
    return users_ingredients

In [4]:
def is_valid_recipe(text):
    lines = text.strip().lower().splitlines()
    if not lines:
        return False
    bad_keywords = [
        "follow me", "on my blog", "thanks for stopping by",
        "i will", "check out", "i'll post", "leave a comment"
    ]
    for keyword in bad_keywords:
        if any(keyword in line for line in lines):
            return False
    # Require at least one bullet point
    if not any(line.strip().startswith("-") or line.strip().startswith("\u2022") for line in lines):
        return False
    return True

In [5]:
def generate_recipe(ingredients, max_attempts=3):
    prompt = (
        f"Create 2 recipes using only these ingredients: {', '.join(ingredients)}.\n"
        "Each recipe should have a title and 3–5 bullet points of instructions. No blog-style writing, just recipes."
    )

    ollama = Client()

    for attempt in range(max_attempts):
        try:
            response = ollama.chat(
                model="llama3.2:latest",  # Directly specify the model here
                messages=[{"role": "user", "content": prompt}]
            )
            text = response['message']['content']
            cleaned = "\n".join(line for line in text.splitlines() if not line.strip().isdigit())

            if is_valid_recipe(cleaned):
                return cleaned.strip()
            else:
                print(f"[Retrying... Attempt {attempt + 1}/{max_attempts}]")
        except Exception as e:
            print(f"Error during attempt {attempt + 1}: {e}")
            print("Retrying...")

    return "Sorry, I couldn't generate a valid recipe. Try running the script again."


In [6]:
def main():
    clear_memory()

    print("Welcome to the AI Recipe Generator!")

    ingredients = get_users_ingredients()
    if not ingredients:
        print("No ingredients provided. Exiting.")
        return

    print("\nGenerating recipe suggestions...\n")
    try:
        recipe = generate_recipe(ingredients)
        print("Here are the recipe suggestions:\n")
        print(recipe)
    except Exception as e:
        print("Error generating recipe:\n")
        print(e)
        print("\nSorry, I couldn't generate a recipe. Please try again.")

    clear_memory()

In [7]:
main()

Welcome to the AI Recipe Generator!
Enter the ingredients you have (type 'end' to finish):

Generating recipe suggestions...

Here are the recipe suggestions:

Recipe 1: Cinnamon Bread

Ingredients:
- 2 eggs
- 1/4 cup sugar
- 2 tablespoons butter, melted
- 1/2 cup flour

Instructions:

• Preheat oven to 350°F (180°C).
• Mix together sugar and melted butter.
• Add eggs one at a time, then add flour and mix until combined.

Recipe 2: Brood Fritters

Ingredients:
- 4 eggs
- 1/4 cup flour
- 1 tablespoon sugar
- 1 tablespoon milk
- 1 tablespoon butter, melted

Instructions:

• Mix together flour, egg, and sugar in a bowl.
• Add milk to the mixture and stir until combined.
• Heat melted butter in a pan over medium heat.
• Drop spoonfuls of the mixture into the hot butter and cook until golden brown.


In [8]:
# import gc
# from nicegui import ui
# from ollama import Client

# def clear_memory():
#     gc.collect()

# def is_valid_recipe(text):
#     lines = text.strip().lower().splitlines()
#     if not lines:
#         return False
#     bad_keywords = [
#         "follow me", "on my blog", "thanks for stopping by",
#         "i will", "check out", "i'll post", "leave a comment"
#     ]
#     for keyword in bad_keywords:
#         if any(keyword in line for line in lines):
#             return False
#     if not any(line.strip().startswith("-") or line.strip().startswith("\u2022") for line in lines):
#         return False
#     return True

# def generate_recipe(ingredients, max_attempts=3):
#     prompt = (
#         f"Create 2 recipes using only these ingredients: {', '.join(ingredients)}.\n"
#         "Each recipe should have a title and 3–5 bullet points of instructions. No blog-style writing, just recipes."
#     )

#     ollama = Client()

#     for attempt in range(max_attempts):
#         try:
#             response = ollama.chat(
#                 model="llama3.2:latest",
#                 messages=[{"role": "user", "content": prompt}]
#             )
#             text = response['message']['content']
#             cleaned = "\n".join(line for line in text.splitlines() if not line.strip().isdigit())
#             if is_valid_recipe(cleaned):
#                 return cleaned.strip()
#             else:
#                 print(f"[Retrying... Attempt {attempt + 1}/{max_attempts}]")
#         except Exception as e:
#             print(f"Error during attempt {attempt + 1}: {e}")
#             print("Retrying...")

#     return "Sorry, I couldn't generate a valid recipe. Try running the script again."

# # ---- NiceGUI UI ----
# messages = ui.column().style("max-width: 800px; margin: auto;")

# @ui.page('/')
# def main_page():
#     clear_memory()
#     ui.label("🍳 AI Recipe Generator").style('font-size: 24px; font-weight: bold; text-align: center;')

#     input_box = ui.input("Enter ingredients (comma-separated)").props('autofocus').style('width: 100%')

#     def handle_submit():
#         user_text = input_box.value.strip()
#         if not user_text:
#             messages.clear()
#             messages.append(ui.label("⚠️ Please enter some ingredients."))
#             return

#         messages.append(ui.label(f"🧑‍🍳 You: {user_text}").style("font-weight: bold;"))
#         ingredients = [i.strip().lower() for i in user_text.split(',') if i.strip()]
#         recipe = generate_recipe(ingredients)
#         messages.append(ui.label(f"🤖 AI:\n{recipe}").style("white-space: pre-wrap;"))

#         input_box.value = ''

#     input_box.on('keydown.enter', lambda e: handle_submit())
#     ui.button("Generate Recipes", on_click=handle_submit)

# ui.run(native=True)
